## The CPA Attack Theory

As a background on the CPA attack, please see the section [Correlation Power Analysis](https://wiki.newae.com/Correlation_Power_Analysis). It's assumed you've read that section and come back to this. Ok, you've done that? Good let's continue.

Assuming you **actually** read that, it should be apparent that there is a few things we need to accomplish:

1. Getting some power traces of our target while it's performing AES encryption.
1. Reading the data, which consists of the analog waveform (trace) and input text sent to the encryption core
1. Making the power leakage model, where it takes a known input text along with a guess of the key byte
1. Implementing the correlation equation, and then looping through all the traces
1. Ranking the output of the correlation equation to determine the most likely key.

This tutorial will deal with both recording power traces using ChipWhisperer and breaking them using a CPA attack.

## Capturing Power Traces
Capturing power traces will be very similar to previous tutorials, except this time we'll be using a loop to capture multiple traces, as well as numpy to store them.

In [ ]:
#import needed chipwhisperer modules
from __future__ import division, print_function

import time
import os

import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm

import chipwhisperer as cw
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
from chipwhisperer.tests.tools_for_tests import FIRMWARE_DIR
from chipwhisperer.capture.api.programmers import XMEGAProgrammer

In [ ]:
#connect to chipwhisperer
scope = cw.scope()
target = cw.target(scope)

In [ ]:
# setup scope parameters
scope.gain.gain = 45
scope.adc.samples = 3000
scope.adc.offset = 1250
scope.adc.basic_mode = "rising_edge"
scope.clock.clkgen_freq = 7370000
scope.clock.adc_src = "clkgen_x4"
scope.trigger.triggers = "tio4"
scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"
scope.io.hs2 = "clkgen"

In [ ]:
%%bash
cd ../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=CW303

In [ ]:
# program the target
programmer = XMEGAProgrammer()
programmer.scope = scope
programmer._logging = None
programmer.find()
programmer.erase()
aes_firmware_dir = os.path.join(FIRMWARE_DIR, 'simpleserial-aes')
aes_hex = os.path.join(aes_firmware_dir, r"simpleserial-aes-CW303.hex")
programmer.program('../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-CW303.hex', memtype="flash", verify=True)
programmer.close()

Below you can see the capture loop. The main body of the loop should look familliar from previous tutorials, except this time we'll be appending to the `traces` list at the end of every loop. At the end, we convert the trace data to numpy arrays, since that's what we'll be using for analysis.

In [ ]:
#Capture Traces
ktp = AcqKeyTextPattern_Basic(target=target)

traces = []
textin = []
keys = []
N = 75  # Number of traces
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    # run aux stuff that should come before trace here

    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
    textin.append(text)
    keys.append(key)

    #target.reinit()

    target.setModeEncrypt()  # only does something for targets that support it
    target.loadEncryptionKey(key)
    target.loadInput(text)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.go()
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    _ = target.readOutput()  # clears the response from the serial port
    traces.append(scope.getLastTrace())

#Convert traces to numpy arrays
trace_array = np.asarray(traces)  # if you prefer to work with numpy array for number crunching
textin_array = np.asarray(textin)
known_keys = np.asarray(keys)  # for fixed key, these keys are all the same

If you want to save your traces for later analysis, here's how:
```python
now = datetime.now()
fmt_string = '{:02}{:02}_{}.npy'
trace_file_path = fmt_string.format(now.hour, now.minute, "traces")
textin_file_path = fmt_string.format(now.hour, now.minute, "textins")
keys_file_path = fmt_string.format(now.hour, now.minute, "keys")

print('Saving results to {},{} and {}...'.format(trace_file_path, textin_file_path, keys_file_path), end='')
# save to a files for later processing
np.save(trace_file_path, trace_array)
np.save(textin_file_path, textin_array)
np.save(keys_file_path, known_keys)
print('Done')
```

In [ ]:
# cleanup the connection to the target and scope
scope.dis()
target.dis()